In [1]:
%matplotlib inline
from python.analysis import Master, vector, Plots
import apps.prod4a_merge_study as merge_study

import awkward as ak
import numpy as np

from rich import print


def GetSharedHitsMask(particleData : Master.ParticleData, e, a, b):
    #! returns the mask for particle b only
    total_mask = None
    for c1, t1 in zip(particleData.channel[e][a], particleData.peak_time[e][a]):
        mask = (c1 == particleData.channel[e][b]) & (t1 == particleData.peak_time[e][b])
        if total_mask is None:
            total_mask = mask
        else:
            total_mask = total_mask | mask
    return total_mask


def GetSharedEnergy(particleData : Master.ParticleData, e, a, b):
    mask_a = GetSharedHitsMask(particleData, e, b, a)
    mask_b = GetSharedHitsMask(particleData, e, a, b)

    energy_a = particleData.hit_energy[e][a][mask_a]
    energy_b = particleData.hit_energy[e][b][mask_b]

    sum_a = ak.sum(energy_a[energy_a > 0])
    sum_b = ak.sum(energy_b[energy_b > 0])
    return sum_a, sum_b

def SharedHitsMetrics(particleData : Master.ParticleData, e, i, mask):
    print(f"number of hits shared with other shower: {ak.count(mask[mask])}")
    shared_hits_energy = particleData.hit_energy[e][i][mask]
    print(f"number of hits shared with valid energy: {ak.count(shared_hits_energy[shared_hits_energy > 0])}")

    hits_channel = particleData.channel[e][i][mask]
    hits_peakTime = particleData.peak_time[e][i][mask]

    shared_hits_energy = particleData.hit_energy[e][i][mask]
    shared_hits_integral = particleData.integral[e][i][mask]
    
    # print(f"{ak.sort(shared_hits_integral)=}")
    # print(f"{ak.sort(shared_hits_energy)=}")
    
    space_points = particleData.space_points[e][i][mask]
    
    # print(f"{ak.sort(space_points.x)=}")
    # print(f"{ak.sort(space_points.y)=}")
    # print(f"{ak.sort(space_points.z)=}")
    
    total_energy = ak.sum(shared_hits_energy[shared_hits_energy > 0])
    total_integral = ak.sum(shared_hits_integral[shared_hits_integral > 0])
    
    # print(f"shared hits total energy: {total_energy}")
    # print(f"shared hits total integral: {total_integral}")

    return {
        "hits_channel"          : hits_channel,
        "hits_peakTime"         : hits_peakTime,
        "shared_hits_energy"    : shared_hits_energy,
        "shared_hits_integral"  : shared_hits_integral,
        "space_points"          : space_points,
        "total_energy"          : total_energy,
        "total_integral"        : total_integral
        }

In [2]:
events = Master.Data("work/ROOTFiles/debug/Prod4a_1GeV_BeamSim_00_Selection.root", nEvents = 10, start = 0)
events.io.ListNTuples()

merge_study.EventSelection(events)
start_showers, to_merge = merge_study.SplitSample(events)
signal, background, signal_all = merge_study.SignalBackground(events, start_showers, to_merge)

[
    'reco_PFP_ID',
    'reco_PFP_Mother',
    'reco_daughter_allShower_sliceID',
    'reco_daughter_allShower_beamCosmicScore',
    'reco_daughter_allSHower_PandoraPDG',
    'pandoraTag',
    'reco_daughter_PFP_emScore_collection',
    'reco_daughter_PFP_trackScore_collection',
    'CNNScore_collection',
    'reco_daughter_allShower_startX',
    'reco_daughter_allShower_startY',
    'reco_daughter_allShower_startZ',
    'reco_daughter_allShower_dirX',
    'reco_daughter_allShower_dirY',
    'reco_daughter_allShower_dirZ',
    'reco_daughter_allShower_coneAngle',
    'reco_daughter_allShower_length',
    'reco_daughter_PFP_nHits_collection',
    'reco_daughter_allShower_energy',
    'reco_daughter_allShower_calibrated_energy',
    'reco_daughter_allShower_hit_channel',
    'reco_daughter_allShower_hit_peakTime',
    'reco_daughter_allSHower_hit_integral',
    'reco_daughter_allShower_hit_energy',
    'reco_daughter_allShower_spacePointX',
    'reco_daughter_allShower_spacePointY',
    'reco_daughter_allShower_spacePointZ',
    'beamNum',
    'reco_beam_sliceID',
    'reco_beam_startX',
    'reco_beam_startY',
    'reco_beam_startZ',
    'reco_beam_endX',
    'reco_beam_endY',
    'reco_beam_endZ',
    'reco_beam_PFP_true_byHits_pdg',
    'reco_beam_PFP_true_byHits_startE',
    'reco_beam_PFP_true_byHits_mass',
    'reco_beam_PFP_true_byHits_startX',
    'reco_beam_PFP_true_byHits_startY',
    'reco_beam_PFP_true_byHits_startZ',
    'reco_beam_PFP_true_byHits_endX',
    'reco_beam_PFP_true_byHits_endY',
    'reco_beam_PFP_true_byHits_endZ',
    'reco_daughter_PFP_true_byHits_ID',
    'reco_daughter_PFP_true_byHits_Mother',
    'reco_daughter_PFP_true_byHits_pdg',
    'reco_daughter_PFP_true_byHits_Mother_pdg',
    'reco_daughter_PFP_true_byHits_startX',
    'reco_daughter_PFP_true_byHits_startY',
    'reco_daughter_PFP_true_byHits_startZ',
    'reco_daughter_PFP_true_byHits_endX',
    'reco_daughter_PFP_true_byHits_endY',
    'reco_daughter_PFP_true_byHits_endZ',
    'reco_daughter_PFP_true_byHits_pX',
    'reco_daughter_PFP_true_byHits_pY',
    'reco_daughter_PFP_true_byHits_pZ',
    'reco_daughter_PFP_true_byHits_startE',
    'reco_daughter_PFP_true_byHits_mass',
    'reco_daughter_PFP_true_byHits_sharedHits',
    'reco_daughter_PFP_true_byHits_sharedHits_collection',
    'reco_daughter_PFP_true_byHits_nHits',
    'reco_daughter_PFP_true_byHits_nHits_collection',
    'reco_daughter_PFP_true_byHits_hitsInRecoCluster',
    'reco_daughter_PFP_true_byHits_hitsInRecoCluster_collection',
    'reco_daughter_PFP_true_byHits_EnergyByHits',
    'reco_daughter_PFP_true_byHits_hit_channel',
    'reco_daughter_PFP_true_byHits_hit_peakTime',
    'reco_daughter_PFP_true_byHits_hit_integral',
    'reco_daughter_PFP_true_byHits_hit_energy',
    'reco_daughter_PFP_true_byHits_spacePointX',
    'reco_daughter_PFP_true_byHits_spacePointY',
    'reco_daughter_PFP_true_byHits_spacePointZ',
    'g4_num',
    'g4_mother',
    'g4_Pdg',
    'g4_startE',
    'g4_mass',
    'g4_startX',
    'g4_startY',
    'g4_startZ',
    'g4_endX',
    'g4_endY',
    'g4_endZ',
    'g4_pX',
    'g4_pY',
    'g4_pZ',
    'Run',
    'SubRun',
    'EventID',
    'totalEvents',
    'beamEvents'
]

/home/sb16165/Documents/pi0-analysis/analysis/python/analysis/Master.py:55: UserWarning: Couldn't apply filters to filters.
  warnings.warn(f"Couldn't apply filters to {var}.")
/home/sb16165/Documents/pi0-analysis/analysis/python/analysis/Master.py:55: UserWarning: Couldn't apply filters to filename.
  warnings.warn(f"Couldn't apply filters to {var}.")
/home/sb16165/Documents/pi0-analysis/analysis/python/analysis/Master.py:55: UserWarning: Couldn't apply filters to _TrueParticleData__pi0_MC.
  warnings.warn(f"Couldn't apply filters to {var}.")


'BeamMCFilter' executed in 0.4749s

number of dalitz decays: 0

'ApplyBeamFilter' executed in 0.4252s

...

False

True

True

True

'SplitSample' executed in 0.3925s

In [2]:
start_shower_indices = ak.local_index(events.recoParticles.number)[np.logical_or(*start_showers)]
print(start_shower_indices)

for i in range(ak.num(start_shower_indices, 0)):
    e1, e2 = GetSharedEnergy(events.trueParticlesBT, i, start_shower_indices[i][0], start_shower_indices[i][1])
    if e1 <= 0 or e2 <= 0: continue
    print("------------------------------------------------------------------------------------------------------")
    print(i)
    print(start_shower_indices[i])
    mask = GetSharedHitsMask(events.trueParticlesBT, i, start_shower_indices[i][0], start_shower_indices[i][1])
    print(f"count: {ak.count(mask[mask])}")
    print(f"shared energies: {e1}, {e2}")
    print(f"difference in shared energy calculation: {e1 - e2}")

    print(f"true pdg: {events.trueParticlesBT.pdg[i][start_shower_indices[i][0]]}, {events.trueParticlesBT.pdg[i][start_shower_indices[i][1]]}")
    true_energy = [events.trueParticlesBT.energy[i][start_shower_indices[i][j]] for j in range(2)]
    print(f"true energies: {true_energy}")
    true_energy_hits = [events.trueParticlesBT.energyByHits[i][start_shower_indices[i][j]] for j in range(2)]
    print(f"true energies by hits: {true_energy_hits}")

    residuals = [true_energy_hits[j] - true_energy[j] for j in range(2)]
    print(f"resiudals: {residuals}")
    print(f"total residual: {residuals[0] + residuals[1]}")
    print(f"total residual after removing shared hit energy: {residuals[0] + residuals[1] - e1}")
    print(f"total residual after removing shared hit energy: {residuals[0] + residuals[1] - e2}")


NameError: name 'events' is not defined

In [4]:
start_shower_indices = ak.local_index(events.recoParticles.number)[np.logical_or(*start_showers)]
print(start_shower_indices)

eve = 1

print(start_shower_indices[eve])

mask_1 = GetSharedHitsMask(events.trueParticlesBT, eve, start_shower_indices[eve][0], start_shower_indices[eve][1])
mask_0 = GetSharedHitsMask(events.trueParticlesBT, eve, start_shower_indices[eve][1], start_shower_indices[eve][0])

c_1 = events.trueParticlesBT.channel[eve][start_shower_indices[eve][1]][mask_1]
c_0 = events.trueParticlesBT.channel[eve][start_shower_indices[eve][0]][mask_0]

t_1 = events.trueParticlesBT.peak_time[eve][start_shower_indices[eve][1]][mask_1]
t_0 = events.trueParticlesBT.peak_time[eve][start_shower_indices[eve][0]][mask_0]

e_1 = events.trueParticlesBT.hit_energy[eve][start_shower_indices[eve][1]][mask_1]
e_0 = events.trueParticlesBT.hit_energy[eve][start_shower_indices[eve][0]][mask_0]

sps_1 = events.trueParticlesBT.space_points[eve][start_shower_indices[eve][1]][mask_1]
sps_0 = events.trueParticlesBT.space_points[eve][start_shower_indices[eve][0]][mask_0]

for h_1, e1, s_1 in zip(zip(c_1, t_1), e_1, sps_1):
    for h_0, e0, s_0 in zip(zip(c_0, t_0), e_0, sps_0):
        if h_0 == h_1 and e0 != e1:
            print(f"{h_0}, {e0}, {s_0}")
            print(f"{h_1}, {e1}, {s_1}")


[[32, 100], [35, 158]]

[35, 158]

(2247, 4451.28857421875), 0.3159021115600794, {x: -999, y: -999, z: -999}

(2247, 4451.28857421875), 0.31535729969001414, {x: -999, y: -999, z: -999}

(2247, 4440.06787109375), 1.7821858282006096, {x: -999, y: -999, z: -999}

(2247, 4440.06787109375), 1.7791122305311584, {x: -999, y: -999, z: -999}

(2209, 4509.42236328125), 1.4070887296341879, {x: -999, y: -999, z: -999}

(2209, 4509.42236328125), 1.4026474766856099, {x: -999, y: -999, z: -999}

In [49]:
def TrueParticleIndices(events, event, particle_number):
    index = ak.local_index(events.trueParticlesBT.number[event])
    return index[events.trueParticlesBT.number[event] == particle_number]


def GetAllSharedEnergy(events : Master.Data, event : int, a : int):
    mask = None
    partner_energy = None
    true_particles = events.trueParticlesBT.GetUniqueParticleNumbers(events.trueParticlesBT.number)

    for i in range(ak.count(true_particles[eve])):
        index = TrueParticleIndices(events, eve, true_particles[event][i])[0]

        if events.trueParticlesBT.number[eve][a] == events.trueParticlesBT.number[eve][index]: continue

        m = GetSharedHitsMask(events.trueParticlesBT, eve, index, a)

        if mask is None:
            mask = m
            partner_energy = ak.unflatten(ak.where(m, events.trueParticlesBT.energy[eve][index], 0), 1, -1)
        else:
            mask = mask | m
            partner_energy = ak.concatenate([partner_energy, ak.unflatten(ak.where(m, events.trueParticlesBT.energy[eve][index], 0), 1, -1)], -1)
    print(f"number of shared hits: {ak.count(mask[mask])}")

    true_energy = events.trueParticlesBT.energy[eve][a]
    weights = true_energy / (true_energy + ak.sum(partner_energy, -1))

    hit_energy = events.trueParticlesBT.hit_energy[eve][a]
    mask = mask & (hit_energy > 0)    
    
    excess_energy = ak.sum(hit_energy[mask] * (1 - weights[mask]))
    return excess_energy

def PrintOut(excess_energy, eve, a):
    print(f"{excess_energy=}")
    print(f"{events.trueParticlesBT.energy[eve][a]=}")
    print(f"{events.trueParticlesBT.energyByHits[eve][a]=}")
    print(f"{events.trueParticlesBT.energyByHits[eve][a] - excess_energy=}")

start_shower_indices = ak.local_index(events.recoParticles.number)[np.logical_or(*start_showers)]
eve = 0

e_0 = GetAllSharedEnergy(events, eve, start_shower_indices[eve][0])
PrintOut(e_0, eve, start_shower_indices[eve][0])
e_1 = GetAllSharedEnergy(events, eve, start_shower_indices[eve][1])
PrintOut(e_0, eve, start_shower_indices[eve][1])


number of shared hits: 74

true_energy=336.7357430563468

weights=<Array [1, 1, 1, 1, 1, ... 0.108, 1, 1, 1, 1] type='667 * float64'>

true_energy=336.7357430563468

mask=<Array [False, False, False, ... False, False] type='667 * bool'>

...

mask=<Array [False, False, False, ... False, False] type='183 * bool'>

excess_energy=272.8673100804525

events.trueParticlesBT.energy=117.31729986975944

events.trueParticlesBT.energyByHits=104.05901579234671

events.trueParticlesBT.energyByHits - excess_energy=-168.8082942881058

In [44]:
weights_0 = events.trueParticlesBT.energy[eve][start_shower_indices[eve][0]] / (ak.sum(partner_energy_0, -1) + events.trueParticlesBT.energy[eve][start_shower_indices[eve][0]])
hit_energy = events.trueParticlesBT.hit_energy[eve][start_shower_indices[eve][0]]

mask = mask_0 & (hit_energy > 0)

excess_energy = hit_energy[mask] * (1 - weights_0[mask])
print(ak.to_list(excess_energy))
excess_energy = ak.sum(excess_energy)
print(excess_energy)
print(events.trueParticlesBT.energy[eve][start_shower_indices[eve][0]])
print(events.trueParticlesBT.energyByHits[eve][start_shower_indices[eve][0]])
print(events.trueParticlesBT.energyByHits[eve][start_shower_indices[eve][0]] - excess_energy)

[
    0.1534386247241889,
    3.255227194475956,
    2.791152132868972,
    3.4611806622051735,
    2.7503094331695994,
    2.5444515691788947,
    2.7724781761194675,
    13.660012694976631,
    20.440726083747876,
    17.136369663584567,
    0.5827363928851576,
    1.4003341221463304,
    0.2844437119557215,
    0.20756762641756926,
    18.50177202595939,
    20.911378138830596,
    1.373851316256424,
    8.448228865405586,
    0.8264083525006478,
    22.829010110946257,
    16.373286517143,
    4.022568041369714,
    18.518688768215572,
    1.1988678318652979,
    3.017052583142241,
    13.452508987115584,
    9.917324823519852,
    21.28403360567601,
    3.0351682368845587,
    17.4792671692784,
    1.708821705089721,
    14.80233651356036,
    0.9457026284192347,
    2.780605770817963
]

272.8673100804525

336.7357430563468

638.8292559160948

365.96194583564227